In [20]:
# Append web folder to system path
import sys
sys.path.append('../web')

In [21]:
from db_config import Base
from Database import Station, Availability, Weather, WeatherPredictive
from sqlalchemy import create_engine, func, Column, String, Integer, Double, Boolean
from sqlalchemy.orm import sessionmaker
import json
import sys
import requests
from datetime import datetime, timedelta

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

In [36]:
# Get the db_info
with open('../dbinfo.json') as f:
    db_info = json.load(f)


USER = db_info['dbConnection']['USER']
PASSWORD = db_info['dbConnection']['PASSWORD']
URI = db_info['dbConnection']['URI']
PORT = db_info['dbConnection']['PORT']
DB = db_info['dbConnection']['DB']
WEATHER_API_KEY = db_info['weatherKey']
FORECAST_URI = 'https://api.openweathermap.org/data/2.5/forecast'


# Create a new session
engine = create_engine(
    'mysql+pymysql://{}:{}@localhost:{}/{}'.format(USER, PASSWORD, PORT, DB), echo=True)
Base.metadata.create_all(bind=engine)
Session = sessionmaker(bind=engine)
session = Session()
print("connected")

2024-04-03 09:07:56,052 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-03 09:07:56,054 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 09:07:56,115 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-03 09:07:56,116 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 09:07:56,133 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-03 09:07:56,135 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 09:07:56,170 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-03 09:07:56,172 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`stations`
2024-04-03 09:07:56,173 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 09:07:56,190 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`availability`
2024-04-03 09:07:56,191 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 09:07:56,209 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`weather`
2024-04-03 09:07:56,210 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 09:07:5

In [24]:
midnight = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

weather_historical = session.query(Weather).filter(
        Weather.time_updated > midnight).all()

# for row in weather_historical:
#     print(row.time_updated, row.wind_speed, row.temperature, row.humidity)

weather_historical_df = pd.DataFrame([row.__dict__ for row in weather_historical])
weather_historical_df = weather_historical_df[['time_updated', 'temperature', 'wind_speed', 'humidity']]
weather_historical_df

2024-04-03 08:56:42,371 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2024-04-03 08:56:42,384 INFO sqlalchemy.engine.Engine SELECT weather.time_updated AS weather_time_updated, weather.type AS weather_type, weather.description AS weather_description, weather.temperature AS weather_temperature, weather.feels_like AS weather_feels_like, weather.min_temp AS weather_min_temp, weather.max_temp AS weather_max_temp, weather.humidity AS weather_humidity, weather.wind_speed AS weather_wind_speed, weather.visibility AS weather_visibility, weather.clouds AS weather_clouds, weather.sunrise AS weather_sunrise, weather.sunset AS weather_sunset 
FROM weather 
WHERE weather.time_updated > %(time_updated_1)s
2024-04-03 08:56:42,387 INFO sqlalchemy.engine.Engine [generated in 0.00305s] {'time_updated_1': datetime.datetime(2024, 4, 3, 0, 0)}


,time_updated,temperature,wind_speed,humidity
0,2024-04-03 00:00:05,8.18,5.66,92
1,2024-04-03 00:30:04,8.18,6.17,91
2,2024-04-03 01:00:05,8.21,6.69,91
3,2024-04-03 01:30:05,8.13,6.17,92
4,2024-04-03 02:00:05,8.18,6.69,93
5,2024-04-03 02:30:05,8.50,5.66,93
6,2024-04-03 03:00:06,8.36,6.17,94
7,2024-04-03 03:30:05,8.46,6.17,93
8,2024-04-03 04:00:05,8.46,6.17,93
9,2024-04-03 04:30:05,8.50,6.17,93


In [37]:
weather_data = requests.get(FORECAST_URI, params={
                                "units": "metric", "lat": 53.344, "lon": -6.2672, "appid": db_info["weatherKey"]})
weather_info = weather_data.json()


In [45]:
weather_historical_df = pd.DataFrame(
    [row.__dict__ for row in weather_historical])
weather_historical_df = weather_historical_df[[
    'time_updated', 'temperature', 'wind_speed', 'humidity', 'type']]

In [55]:
for i in predicted_weather_df['weather'][0]:
    print(i)


{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}


In [59]:
predicted_weather_df['weather'][10]

[{'id': 804,
  'main': 'Clouds',
  'description': 'overcast clouds',
  'icon': '04d'}]

In [60]:
station_id = 2

# use pd.DataFrame because data is already an object
predicted_weather_df = pd.DataFrame(weather_info['list'])
predicted_weather_df['temperature'] = [row['main']['temp'] for row in weather_info['list']]
predicted_weather_df['humidity'] = [row['main']['humidity'] for row in weather_info['list']]
predicted_weather_df['wind_speed'] = [row['wind']['speed'] for row in weather_info['list']]
predicted_weather_df['type'] = [row['weather'][0]['main']
                                    for row in weather_info['list']]
predicted_weather_df['time_updated'] = pd.to_datetime(predicted_weather_df['dt_txt'])

predicted_weather_df['humidity'] = predicted_weather_df['humidity'].astype('int64')

# Convert temperature and wind_speed to float64
predicted_weather_df['temperature'] = predicted_weather_df['temperature'].astype('float64')
predicted_weather_df['wind_speed'] = predicted_weather_df['wind_speed'].astype('float64')


predicted_weather_df = predicted_weather_df[['time_updated', 'temperature', 'wind_speed', 'humidity', 'type']]

predicted_weather_df.head()



,time_updated,temperature,wind_speed,humidity,type
0,2024-04-03 09:00:00,7.55,4.88,94,Rain
1,2024-04-03 12:00:00,8.00,4.37,91,Rain
2,2024-04-03 15:00:00,8.85,1.73,86,Rain
3,2024-04-03 18:00:00,8.23,2.52,89,Clouds
4,2024-04-03 21:00:00,7.05,2.34,97,Rain


In [29]:
weather_combined = pd.concat([weather_historical_df, predicted_weather_df])
weather_combined.head()

,time_updated,temperature,wind_speed,humidity
0,2024-04-03 00:00:05,8.18,5.66,92
1,2024-04-03 00:30:04,8.18,6.17,91
2,2024-04-03 01:00:05,8.21,6.69,91
3,2024-04-03 01:30:05,8.13,6.17,92
4,2024-04-03 02:00:05,8.18,6.69,93


In [30]:
current_date = datetime.now()

# Generate a list of hours for today
hours_today = [current_date.replace(hour=h, minute=30, second=0, microsecond=0) for h in range(24)]
hourly_df = pd.DataFrame(hours_today, columns=['time_updated'])

hourly_df.head()

,time_updated
0,2024-04-03 00:30:00
1,2024-04-03 01:30:00
2,2024-04-03 02:30:00
3,2024-04-03 03:30:00
4,2024-04-03 04:30:00


In [31]:
df = pd.merge_asof(hourly_df, weather_combined, on='time_updated')
df.head()

,time_updated,temperature,wind_speed,humidity
0,2024-04-03 00:30:00,8.18,5.66,92
1,2024-04-03 01:30:00,8.21,6.69,91
2,2024-04-03 02:30:00,8.18,6.69,93
3,2024-04-03 03:30:00,8.36,6.17,94
4,2024-04-03 04:30:00,8.46,6.17,93


In [32]:
days = ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
        'Wednesday']

# One hot encode day of the week
df['weekday'] = df['time_updated'].dt.day_name()
df['hour'] = df['time_updated'].dt.hour
for day in days:
    df[day] = df['weekday'] == day

weather_types = ['Clear', 'Clouds', 'Drizzle', 'Mist', 'Rain']

for weather_type in weather_types:
    df[weather_type] = df['type'] == weather_type

df.drop('time_updated', axis=1, inplace=True)
df.drop('weekday', axis=1, inplace=True)
df.drop('type', axis=1, inplace=True)

KeyError: 'type'

In [ ]:
df

,temperature,wind_speed,humidity,hour,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,predicted_available
0,6.55,2.06,93,0,False,False,False,False,False,True,False,12.303959
1,6.31,2.57,93,1,False,False,False,False,False,True,False,11.749343
2,5.64,3.09,93,2,False,False,False,False,False,True,False,11.271157
3,6.44,1.03,92,3,False,False,False,False,False,True,False,16.543342
4,5.89,1.54,92,4,False,False,False,False,False,True,False,15.082775
5,4.82,2.06,93,5,False,False,False,False,False,True,False,13.445218
6,4.52,2.06,93,6,False,False,False,False,False,True,False,13.704222
7,4.67,1.54,94,7,False,False,False,False,False,True,False,15.102604
8,5.49,0.51,93,8,False,False,False,False,False,True,False,18.948694
9,5.51,0.51,93,9,False,False,False,False,False,True,False,19.350664
